# LZ77-Kompression

## Eingabedaten

In [1]:
InputSelect = 1

In [ ]:
match InputSelect:
    case 1:
        Input = 'abracabrabra'
        LookAheadLength, SearchLength, Output = 4, 6, True
    case 2:
        Input = 'aacaacabcabaaac'
        LookAheadLength, SearchLength, Output = 9, 12, True
    case 3:
        Input = 'aaaaa-aaaa-aaa-aa-a-abc-bca-a-aa-aaa-aaaa-aaaaa'
        LookAheadLength, SearchLength, Output = 10, 10, False

SyntaxError: invalid syntax (3016325489.py, line 1)

## Pufferspeicher

### Zeichen für Ende der Eingabe

In [3]:
InputEnd = '.'

### Kombination von Wörterbuch (Search Buffer) und Vorschaupuffer (Look Ahead Buffer)

In [4]:
InputLength = len (Input)

In [5]:
Buffer = InputEnd * SearchLength + Input + InputEnd * (LookAheadLength + 1)
print (Buffer)

......abracabrabra.....


## Suchfenster

### initialisieren

In [6]:
def Init ():
    global WindowOrigin, WindowMaximum
    WindowOrigin = SearchLength
    WindowMaximum = WindowOrigin + InputLength

Init()

### am Ende der Eingabe?

In [7]:
def AtEnd ():
    return WindowOrigin >= WindowMaximum

### bewegen

In [8]:
def Move (increment):
    global WindowOrigin
    WindowOrigin += increment

### Zeichen lesen

In [9]:
def Character (i):
    return Buffer [WindowOrigin + i]

### Zeichenkette lesen

In [10]:
def String (i, k):
    return Buffer [WindowOrigin + i : WindowOrigin + k]

### ausgeben

In [11]:
def Print ():
    if Output:
        frm = '{:>3}'
        spc = ' ' * 2
        sep = '|'
        for i in range (-SearchLength, LookAheadLength):
            if i == 0:
                print (sep, end = spc)
            print (frm.format (i), end = spc)
        print()
        for i in range (-SearchLength, LookAheadLength):
            if i == 0:
                print (sep, end = spc)
            print (frm.format (Character (i)), end = spc)
        print()

Print()

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    .    .    .    .    .  |    a    b    r    a  


## Kompression

In [12]:
def Search ():
    for n in range (LookAheadLength - 1, 0, -1):
        s = String (0, n)
        for i in range (-SearchLength, 0):
            t = String (i, i + n)
            if (s == t):
                return -i, n
    return 0, 0

In [13]:
def CompressPrint1 ():
    if Output:
        Print()

In [14]:
def CompressPrint2 (tup):
    if Output:
        print ('\n--->  ', tup, '\n')

In [15]:
def Compress ():
    code = []
    Init()
    while not AtEnd():
        CompressPrint1()
        i, n = Search()
        tup = (i, n, Character (n))
        Move (n + 1)
        CompressPrint2 (tup)
        code.append (tup)
    CompressPrint1()
    return code

comp = Compress()

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    .    .    .    .    .  |    a    b    r    a  

--->   (0, 0, 'a') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    .    .    .    .    a  |    b    r    a    c  

--->   (0, 0, 'b') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    .    .    .    a    b  |    r    a    c    a  

--->   (0, 0, 'r') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    .    .    a    b    r  |    a    c    a    b  

--->   (3, 1, 'c') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  .    a    b    r    a    c  |    a    b    r    a  

--->   (5, 3, 'a') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  a    c    a    b    r    a  |    b    r    a    .  

--->   (3, 3, '.') 

 -6   -5   -4   -3   -2   -1  |    0    1    2    3  
  r    a    b    r    a    .  |    .    .    .    .  


In [16]:
for tup in comp:
    print (tup)

(0, 0, 'a')
(0, 0, 'b')
(0, 0, 'r')
(3, 1, 'c')
(5, 3, 'a')
(3, 3, '.')


## Dekompression

In [17]:
def DecompressPrint1 (i, n, character):
    if Output:
        print ('{:2} {:2} {:>2}' .format (i, n, character), end = ' ' * 5)

In [18]:
def DecompressPrint2 (string):
    if Output:
        print (string)

In [19]:
def Decompress (code):
    string = ''
    for k, n, character in code:
        DecompressPrint1 (k, n, character)
        i = len (string) - k
        while n > 0:
            m = min (k, n)
            string += string [i : i + m]
            n -= m
        if character != InputEnd:
            string += character
        DecompressPrint2 (string)
    return string

decomp = Decompress (comp)
print (decomp)

 0  0  a     a
 0  0  b     ab
 0  0  r     abr
 3  1  c     abrac
 5  3  a     abracabra
 3  3  .     abracabrabra
abracabrabra


In [20]:
print (decomp == Input)

True
